### Code to train Mask-NoMask Classifier ###

#### 1. Load all images from training folder
#### 2. Create array of images and labels
#### 3. Load and modify MobileNet Model
#### 4. Re-train the model on training data

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt

import numpy as np
import os
import pandas as pd
import cv2

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
alpha = 1e-4
epoch = 30
batch_size = 32

# paths:
training_folder = '/Users/apple/bhavik/COS80026/Project/Final submission/GitHub/Data/dataset_mask_no_mask/'

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images

print("[INFO] loading images...")
imagePaths = list(paths.list_images(training_folder))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
   
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

# convert the data and labels to NumPy arrays

data = np.array(data, dtype="float32")
labels = np.array(labels)
df_labels = pd.DataFrame(labels)
df_labels[0] = df_labels[0].map({"without_mask":0,"mask":1})
labels = np.array(df_labels[0])


In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=np.random)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model

import tensorflow as tf

Model1 = baseModel.output
Model1 = AveragePooling2D(pool_size=(7, 7))(Model1)
Model1 = Flatten(name="flatten")(Model1)
Model1 = Dense(128, activation="relu")(Model1)
Model1 = Dropout(0.5)(Model1)
Model1 = Dense(2, activation=tf.nn.softmax)(Model1)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=Model1)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False


In [ ]:

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=alpha, decay=alpha / epoch)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H = model.fit(
    aug.flow(trainX, trainY, batch_size=batch_size),
    steps_per_epoch=len(trainX) // batch_size,
    validation_data=(testX, testY),
    validation_steps=len(testX) // batch_size,
    epochs=epoch)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=batch_size) #32
print(predIdxs)


# serialize the model to disk
print("[INFO] saving mask detector model...")
modelname = models + "/" + "mask_type_model_v1"
model.save(modelname, save_format="h5")


In [ ]:
predictions = np.argmax(predIdxs,axis=1)
print(predictions)

sklearn.metrics.confusion_matrix(testY,predictions)